In [ ]:
import pandas as pd
import numpy as np
import time

# 1️⃣ Vectorize "easy" operations with pandas and numpy

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/e/ee/Vectorized-addition.gif/220px-Vectorized-addition.gif'>

In [ ]:
n_rows = 5000
n_cols = 5000
a = np.random.randint(low=0, high=5, size=(n_rows,n_cols), dtype=np.int64)
df = pd.DataFrame(a)
df

Let's consider different methods for summing all elements of the matrix

In [ ]:
%%time

# Worse case scenario: double python for loop (O(n^2)) !!
sum = 0
for row_id in range(n_rows):
    for el in a[row_id,:]:
        sum = sum + el
sum

## 1.1) Numpy

In [ ]:
%%time
a.sum()

In [ ]:
%%time
# Column-wise operation (axis=0): Apply np.sum 5000 times for each columns, then apply np.sum one last time to get the final result!
np.apply_along_axis(np.sum, 0, a).sum()

In [ ]:
%%time
# Row-wise operation (axis=1): Apply np.sum 5000 times for rows columns, then apply np.sum one last time to get the final result!
np.apply_along_axis(np.sum, 1, a).sum()

❗️ Numpy is row-major (by default). Avoid iterating on its columns

<img src='https://craftofcoding.files.wordpress.com/2017/02/rowcolumnarrays.jpg?w=620&h=356'>

Actually, we can change its numpy's storage format from row to columns!

In [ ]:
a           = np.ones((n_rows,n_cols), order='C') # C-style BY DEFAULT
a_col_major = np.ones((n_rows,n_cols), order='F') # F for Fortran style

In [ ]:
%%time
# Column-wise operation
np.apply_along_axis(np.sum, 0, a_col_major).sum()

In [ ]:
%%time
# Row-wise operation
np.apply_along_axis(np.sum, 1, a_col_major).sum()

## 1.2)🐼 Pandas

In [ ]:
%%time
# Using pandas vectorized sum is much faster (here, it is called n_cols + 1 times)
df.sum().sum()

❗️**Pandas is column-major**. **Avoid iterating manually on its rows** (convert to numpy which is row-major if needs be)  

In [ ]:
# Get column named "0", 1000 times
%timeit -n1000 df[0]

In [ ]:
# Get first ROW, 1000 times
%timeit -n1000 df.iloc[0,:]

## 1.3) What about more complex operations?

In [ ]:
N = 100000
A_list = np.random.randint(0, 4, N)
B_list = np.random.randint(0, 4, N)
df = pd.DataFrame({'A': A_list, 'B': B_list})
df

Imagine we want to do divide col "A" by col "B" in the following matrix, but without any `NaN`  

Remember, in numpy
- `1./0. == np.inf` ✅
- but `0./0. == np.nan` 😰


In [ ]:
%%time
# This would be super fast but it's not what we want!
_ = df['A']/df['B']
_.isna().sum()

In [ ]:
# Let's define our custom division function

def divide_without_nan(row):
    if row[0] == row[1]:
        return 1
    return float(row[0]/row[1])

We could "apply" on each row

In [ ]:
%%time
new = df.apply(lambda row: divide_without_nan(row), axis=1)
assert new.isna().sum() == 0

☝️ Remember, we need to try to avoid looping over pandas rows.

❓**Try to make it faster by converting to it to numpy first** (which is row-major), then use `np.apply_along_axis()`!
You should find it's about 4 time faster!

<details>
  <summary markdown='span'>🎁 Solution</summary>

```python
%%time
new = np.apply_along_axis(divide_without_nan, 1, df.values)
assert np.isnan(new).sum() == 0
```

</details>

In [ ]:
# YOUR CODE HERE

We are still very very far from the purely "vectorized" speed offered by C-based numpy/pandas operation!  
❓ **Can you find a way to make this computation fully vectorized ?**

<details>
    <summary markdown='span'>💡Hints</summary>

`np.where()`
</details>

In [ ]:
# YOUR CODE HERE

# 2️⃣ When you can't find "vectorized" numpy equivalents?

In [ ]:
url = 'https://wagon-public-datasets.s3.amazonaws.com/taxi-fare-ny/train_500k.csv'
df = pd.read_csv(url)
df.head()

In [ ]:
from math import radians, sin, cos, asin, sqrt
EARTH_RADIUS = 6371

def haversine(lon1: float,lat1: float,lon2: float,lat2: float) -> float:
    """returns the geodistance in km between two tuple of coordinates"""
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * EARTH_RADIUS * asin(sqrt(a))

## 2.1) Baseline score: python loops

In [ ]:
%%time
distances = df.apply(lambda row: haversine(row["pickup_longitude"], row["pickup_latitude"], row["dropoff_longitude"], row["dropoff_latitude"]), axis=1)

😵 What did we just do! Iterating on DataFrame by rows is a very bad practice...

👇 We can do a bit better by transposing dataframe and iterate on its **columns**...

In [ ]:
%%time
df_coor = df[["pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"]]
distances = df_coor.T.apply(lambda col: haversine(*col), axis=0, raw=True)

But we're still applying a non-vectorized lambda function, so pandas/numpy are not helping at all!

Let's get rid of pandas and numpy overhead altogether and see what happens in pure python...

In [ ]:
def haversine_loop_python(coordinates: list) -> list:
    distances = np.zeros(len(coordinates))
    for i, row in enumerate(coordinates):
        distance = haversine(*row)
        distances[i] = distance
    return distances

In [ ]:
%time distances = haversine_loop_python(list(df_coor.values))

☝️ Better, but still way too long! Basically, **a for loop in python is almost always sub-optimal**

Let's vectorize this code!

## 2.2) Numba

### first option: `jit` your outer python loop directly

If you don't have a particular function to optimize, but a whole code block with python loops, you can use `@jit` to let numba try to create a optimized version of your whole code block the first time it runs. Then, all subsquent runs will be vectorized

In [ ]:
from numba import jit

In [ ]:
@jit
def haversine_loop_jit(coordinates):
    distances = np.zeros(len(coordinates))
    for i, row in enumerate(coordinates):
        lon1, lat1, lon2, lat2 = map(radians, [row[0], row[1], row[2], row[3]])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
        distance = 2 * EARTH_RADIUS * asin(sqrt(a))
        distances[i] = distance
    return distances

In [ ]:
# Run this twice
%time distances = haversine_loop_jit(df_coor.values)

☝️ Amazing, isn't it! Another 10x improvement here

#### second option: `vectorize` your inner scalar computation, then call it with numpy arrays instead of scalars!

Syntax is simply 

```python
@vectorize([output_type(arg1_type, arg2_type...)])
def my_vectorized_python_function(arg1, arg2..):
    pass
```
☝️ You just have to specify to numba what are the input and output type you expect, and the **new function can now take iterables** like pd.Series or np.ndarray instead of scalar!

In [ ]:
from numba import vectorize, float64

In [ ]:
@vectorize([float64(float64, float64, float64, float64)])
def vectorized_haversine(lon1,lat1,lon2,lat2):
    """returns the geodistance in km between two tuple of coordinates"""
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * EARTH_RADIUS * asin(sqrt(a))

In [ ]:
# Run this cell twice to see best speed performance
%time distances = vectorized_haversine(df["pickup_longitude"], df["pickup_latitude"], df["dropoff_longitude"], df["dropoff_latitude"])

If you had multiple CPUs or GPUs, you could go even faster with 

```python
@vectorize(target="parallel") # Multi-core CPU
@vectorize(target="CUDA") # GPU acceleration
```


📺 If you are not convinced yet, take 5min to see [this real-world example](https://youtu.be/x58W9A2lnQc?t=719) on youtube

## 2.3) Cython

If you know how to write C, you can also code your own vectorized function using Cython. The goal is to minimize the amount of "python" code (in yellow)

In [ ]:
%load_ext Cython

In [ ]:
%%cython -a

from libc.math cimport sin, cos, asin, sqrt
   
## Equivalent to 3.1415927 / 180
cdef float PI_RATIO = 0.017453293

cdef float deg2rad(float deg):
    cdef float rad = deg * PI_RATIO
    return rad
    
def cython_haversine(float lon1, float lat1, float lon2, float lat2):
    cdef float rlon1 = deg2rad(lon1)
    cdef float rlon2 = deg2rad(lon2)
    cdef float rlat1 = deg2rad(lat1)
    cdef float rlat2 = deg2rad(lat2)
    
    cdef float dlon = rlon2 - rlon1
    cdef float dlat = rlat2 - rlat1

    cdef float a = sin(dlat/2)**2 + cos(rlat1) * cos(rlat2) * sin(dlon/2)**2

    cdef float c = 2 * asin(sqrt(a))
    cdef float km = 6371 * c
    return km


In [ ]:
def haversine_loop_cython(coordinates: list) -> list:
    distances = np.zeros(len(coordinates))
    for i, row in enumerate(coordinates):
        distance = cython_haversine(*row)
        distances[i] = distance
    return distances

In [ ]:
%time distances = haversine_loop_python(df_coor.values)

In [ ]:
%time distances = haversine_loop_cython(df_coor.values)

In [ ]:
%time distances = haversine_loop_jit(df_coor.values)

❓ **Why isn't `haversine_loop_cython` faster than jit?**

<details>
  <summary markdown='span'>🎁 Answer</summary>

Because we have only coded the scalar function `cython_haversine` in C, not the whole haversine_loop.  

@jit, on the other hand, has had the opportunity to "see" and optimize the whole outer-loop.

</details>

# 3️⃣ GPUs?

👉 Open this notebook with [Google Colab](https://colab.research.google.com/), then and choose Runtime --> Runtime type: **GPU**

We'll use Tensorflow to multiply matrices much faster than numpy!

In [ ]:
import tensorflow as tf
import numpy as np

You can manually select the processor on which to perform your tensor operations.

⏩⏩⏩ Check out the [documentation](https://www.tensorflow.org/guide/gpu).

In [ ]:
# Check CPU's available
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))

In [ ]:
# If you've set up Colab correctly, you should have a GPU avaiable.
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# Matrix multiplication function performing operation and returning us the time.
import time

def time_matmul(types,x):

    start = time.time() 

    if types=='numpy':
        np.matmul(x,x)

    else:
        tf.matmul(x,x)

    diff = time.time() - start

    return diff*1000


👇 Run this cell twice (the first time it runs, tensorflow compilation for GPU takes a bit of time)

In [ ]:
shape_dim = []
num_time = []
cpu_tf_time = []
gpu_tf_time = []

for shape in range(500,2001,100):

    print(f"Multiplication of shape [{shape},{shape}]")

  # Start with shape 500,500 to 2000,2000 with an increase of 100
    shape_dim.append(shape)

  # Numpy multiplication
    x_np = np.random.uniform(size=[shape,shape])
    num_time.append(time_matmul('numpy',x_np))
  
  #Tensor in CPU
    with tf.device("CPU:0"):
        x = tf.random.uniform([shape, shape])
        cpu_tf_time.append(time_matmul('cpu',x))
        
  #Tensor in GPU multiplication
    with tf.device("GPU:0"): #Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
        x = tf.random.uniform([shape, shape])
        gpu_tf_time.append(time_matmul('gpu',x))

print("Done multiplying!")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(shape_dim, num_time, label="Numpy Array")
plt.plot(shape_dim, cpu_tf_time, label="Tensor in CPU")
plt.plot(shape_dim, gpu_tf_time, label="Tensor in GPU")
plt.grid()
plt.xlabel("Shape of the Matrix")
plt.ylabel("Time in milliseconds")
plt.legend()